# Check GPU

In [1]:
#Check for GPU
import tensorflow as tf
# if tf.test.gpu_device_name():
#     print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
# else:
#     print('Please install GPU version of TF')
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [2]:
#check keras and tf versions
import keras
print(keras.__version__)
import tensorflow as tf
print(tf.__version__)

2.4.3
2.2.0


# Cochleagram Generation

Loads an audio file and generates a cochleagram using the tfcochleagram.py library. 



In [3]:
from __future__ import division
# For displaying audio and images in notebook
#import IPython.display as ipd

from PIL import Image
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy
import time
import os
import sys
from os import listdir
from os.path import isdir, join, dirname, join, realpath
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import tfcochleagram
import h5py
import math
# Helper functions for loading audio
from utils import *


In [4]:
#this is the 4d padding function adapted from Kevin's 2d version
def pad(data_4d, ref_shape, pad_val=-9999):
    '''
    Pads a 4D matrix "data" with the pad_value to make it have shape ref_shape.
    All the padded values occur "off the end" of the data array, so that the
    original array occupies the upper left of the resulting matrix.
    Refer to pad_nans method to pad with nan values
    '''
    padded_data = pad_val*np.ones(ref_shape)
    padded_data[:, :data_4d.shape[1], :data_4d.shape[2], :] = data_4d
    return padded_data

In [5]:
#Here we calculate the longest time and store it into max len
t0 = time.time()
cwd = os.getcwd()
PATH = join(cwd, 'Words')

onlydirs = [f for f in listdir(PATH) if isdir(join(PATH, f))]

total_len = []

for dirs in onlydirs:
    dirname = PATH + "/" + dirs 
    rfnArraymax = [os.path.join(dirname, f)for f in os.listdir(dirname) if f.endswith('.wav')]
    
    for f in rfnArraymax:
        test_audio, SR = load_audio_wav_resample(f, DUR_SECS='full', resample_SR=16000)
        total_len.append(math.ceil(len(test_audio)/80))
        #print(math.ceil(len(test_audio)/80))
max_len = np.max(total_len)
print(max_len)

251


In [6]:
#Write cochs to individual np arrays
j = 0
for dirs in onlydirs:
    dirname = PATH + "/" + dirs 
    rfnArray = [os.path.join(dirname, f)for f in os.listdir(dirname) if f.endswith('.wav')]
    i = 0
    
    for f in rfnArray:
        j = j + 1
        print(j, end='\r')
        #print('Running demo with sound file: %s ' % f)
        test_audio, SR = load_audio_wav_resample(f, DUR_SECS='full', resample_SR=20000)
        # Generally a good idea to rms normalize the audio
        test_audio = rms_normalize_audio(test_audio, rms_value=0.01)
        # Using rFFT below, and it is currently implemented only for even # signals. 
        if len(test_audio.ravel())%2:
            test_audio = test_audio[:-1]
            #print(test_audio)
        if len(test_audio.shape) == 1: # we need to make sure the input node has a first dimension that corresponds to the batch size
            test_audio = np.expand_dims(test_audio,0) 
        nets = {}
        # tfcochleagram expects a dictionary with 'input_signal' defined for the input audio
        nets['input_signal'] = tf.Variable(test_audio, dtype=tf.float32)
        nets = tfcochleagram.cochleagram_graph(nets, SR, rFFT=True)
        #with tf.Session() as sess:
        #with tf.compat.v1.Session() as sess:
        nets['input_signal'] = test_audio
        cochleagram = nets['cochleagram']
        filters_out = nets['filts_tensor']
        
        #this is where padding happening
        cochleagram = pad(cochleagram, (1,211,max_len,1), -9999) #211 is the channels in the coch.
        cochleagram = cochleagram[:,:,:,0] #here we get rid of the last axis because we do not need depth
        cochleagram = np.rollaxis(cochleagram,1,3) # here we move the second axis (1) 3 times to make it totalXtimeXchannels
        #print(cochleagram.shape)
        
        #save the cochs into pickles
        MAIN_PATH = join(cwd, 'data')
        #if isdir(MAIN_PATH + '/%s' %dirs) == False:  #for now we do not want to create subfolders    
        # os.mkdir(MAIN_PATH + '/%s' %dirs)
        
        filenames_with_extension = os.listdir(dirname)
        
        filenames=[x.split('.')[0] for x in filenames_with_extension]
        filename=filenames[i]
        i += 1
        #filex=os.path.basename(filename)       

        #write to numpy array
        #f = MAIN_PATH + '/%s' %dirs + '/' + filename #no subfolers under data folder
        f = MAIN_PATH + '/' + filename
        np.save(f,cochleagram[0,:,:])
        #write to pickle
        #import pickle
        #with open(MAIN_PATH + '/%s' %dirs + '/' + filename +'.PICKLE', 'wb') as f:
        #    pickle.dump(cochleagram[:,:,:], f, protocol= 4)
        
t1 = time.time()
timer = t1-t0
print(timer)

3852.8852260112762
